In [16]:
#Importing Libraries
import nltk
from nltk.stem.lancaster import LancasterStemmer
from PIL import Image as pil
import tflearn
import tensorflow as tf
tf.compat.v1.disable_v2_behavior
import numpy as np
import random
import json
import pickle

stemmer = LancasterStemmer()

In [3]:
with open('intents.json') as data:
    intents = json.load(data)

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #Tokenizing each word and adding to word list
        w = nltk.word_tokenize(pattern,language="english", preserve_line=False)
        words.extend(w)
        documents.append((w, intent['tag']))
        #Adding to classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

#Stemming each word in lower and removing duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

#Removing duplicates
classes = sorted(list(set(classes)))

print('Documents: ',len(documents))
print('Classes: ',len(classes),classes)
print(len(words),"Stemmed Words",words)

Documents:  27
Classes:  9 ['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']
48 Stemmed Words ["'d", "'s", 'a', 'acceiv', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'do', 'doe', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'lat', 'lik', 'mastercard', 'mop', 'of', 'on', 'op', 'rent', 'see', 'tak', 'thank', 'that', 'ther', 'thi', 'to', 'today', 'we', 'what', 'when', 'which', 'work', 'yo', 'you']


In [ ]:
#Creating training data
training = []
output = []
output_empty = [0] * len(classes)

#Training set, bag of words for each sentence
for doc in documents:
    #Initializing bag of words
    bag = []
    pattern_words = doc[0]
    #Stemming each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    #Creating a one hot encoded vector with '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

#Shuffling features and turning into np.array
random.shuffle(training)

#Splitting training data into features and labels
train_x = [entry[0] for entry in training]
train_y = [entry[1] for entry in training]

train_x = np.array(train_x, dtype=np.float32)
train_y = np.array(train_y, dtype=np.float32)

#print(train_x[0])
#print(train_y[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[0. 0. 0. 1. 0. 0. 0. 0. 0.]


In [13]:
#Reset underlying graph data
tf.compat.v1.reset_default_graph()

#Building neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

#Defining model and setting up tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

#Applying gradient descent
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.05011 | time: 0.014s
| Adam | epoch: 1000 | loss: 0.05011 - acc: 0.9986 -- iter: 24/27
Training Step: 4000  | total loss: 0.04792 | time: 0.018s
| Adam | epoch: 1000 | loss: 0.04792 - acc: 0.9988 -- iter: 27/27
--


INFO:tensorflow:c:\Users\Diya\OneDrive\Desktop\Programs\Chatbot\Chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [17]:
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )